In [ ]:
import numpy as np 
import pandas as pd 
import os
import re

In [ ]:
path = os.path.join(os.getcwd(),"Data")
revenuepath = os.path.join(path,"Revenue")
stockspath = os.path.join(path,"Stock")
print(stockspath)
print(revenuepath)
print(path)

In [ ]:
rev = pd.read_csv(os.path.join(revenuepath,"500002.csv"))
stk = pd.read_csv(os.path.join(stockspath,"500002.csv"))
stk["Date"] = pd.to_datetime(stk["Date"])

In [ ]:
s = min(rev.year)
e = max(rev.year)

In [ ]:
stk['Revenue'] = ''
stk['Income'] = ''
stk['Expenditure'] = ''
stk['Net Profit'] = ''
stk['EPS'] = ''
cols = ['Revenue','Income','Expenditure','Net Profit','EPS']
rep = ['net sales/revenue from operations','total income','expenditure','net profit','eps']

In [ ]:
for index,row in stk.iterrows():
    q = (row.Date.month-1)//3 + 1
    samp = rev[(rev['year']==row.Date.year)&(rev['quartile']==q)]
    if samp.shape[0] !=0:
        stk.loc[index,cols] = samp.iloc[0][rep].values
    else:
        stk.loc[index,cols] = [0,0,0,0,0]

In [ ]:
stk['year'] = pd.DatetimeIndex(stk['Date']).year
stk = stk[(stk.year >= s)&(stk.year <= e) & stk["Revenue"] !=0 ]
stk = stk.drop(["year"],axis=1)

In [ ]:
bands = [2,4,8]

In [ ]:
for band in bands:
    stk['Revenue last '+str(band)+' quarters'] = ''
    stk['Income last '+str(band)+' quarters'] = ''
    stk['Expenditure  last '+str(band)+' quarters'] = ''
    stk['Net Profit  last '+str(band)+' quarters'] = ''
    stk['EPS last '+str(band)+' quarters'] = ''
    bcols = ['Revenue last '+str(band)+' quarters','Income last '+str(band)+' quarters','Expenditure  last '+str(band)+' quarters','Net Profit  last '+str(band)+' quarters','EPS last '+str(band)+' quarters']

    for index,row in stk.iterrows():
        q = (row.Date.month-1)//3 + 1
        samp = rev[(rev['year']==row.Date.year)&(rev['quartile']==q)]
        r = samp.index.values[0]
        if r+band+1 < rev.shape[0]:
            v = range(r+1,r+band+1)
            stk.loc[index,bcols] = rev.loc[v,rep].sum().values

In [ ]:
stk["p/e"] = stk["Close Price"]/stk["EPS"]

In [ ]:
stk.to_csv(os.path.join(stockspath,"rp500002.csv"),index=None)